In [1]:
import csv

reader = []

with open('airports.csv', 'rb') as filename:
    reader = list(csv.reader(filename))
    
    
    #for row in reader:
        #if row[2] == 'large_airport':
            #print row[3]
            
    for key, value in zip ( reader[0], reader [27863]):
        print key, value
        
    print reader[0]

id 4154
ident LFMK
type medium_airport
name Carcassonne Airport
latitude_deg 43.215999603271484
longitude_deg 2.3063199520111084
elevation_ft 433
continent EU
iso_country FR
iso_region FR-K
municipality Carcassonne/Salvaza
scheduled_service yes
gps_code LFMK
iata_code CCF
local_code 
home_link 
wikipedia_link http://en.wikipedia.org/wiki/Carcassonne_Salvaza_Airport
keywords 
['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg', 'elevation_ft', 'continent', 'iso_country', 'iso_region', 'municipality', 'scheduled_service', 'gps_code', 'iata_code', 'local_code', 'home_link', 'wikipedia_link', 'keywords']


In [2]:
lon,lat = 52.6541, -7.2448

def dist(x,y):
    lon,lat = 52.6541, -7.2448
    return abs(lon -float(x)) + abs(lat - float(y)) < 1

for row in reader[1:]:
    if row[8] == 'FR' and row[2] in ['large_airport', 'medium_airport'] and row[11]=='yes':
        print ( ", ".join ( row[x] for x in [0,9,3,4,5,7,8,10]) )
            #print " ".join(row)

311396, FR-L, Brive Souillac Airport, 45.039722, 1.485556, EU, FR, 
4058, FR-B, Agen-La Garenne Airport, 44.17470169067383, 0.5905560255050659, EU, FR, Agen/La Garenne
4060, FR-B, Bordeaux-Mérignac Airport, 44.828300476100004, -0.715556025505, EU, FR, Bordeaux/Mérignac
4061, FR-B, Bergerac-Roumanière Airport, 44.82529830932617, 0.5186110138893127, EU, FR, Bergerac/Roumanière
4064, FR-T, La Rochelle-Île de Ré Airport, 46.17919921875, -1.1952799558639526, EU, FR, La Rochelle/Île de Ré
4065, FR-T, Poitiers-Biard Airport, 46.58769989013672, 0.30666598677635193, EU, FR, Poitiers/Biard
4067, FR-L, Limoges Airport, 45.86280059814453, 1.1794400215148926, EU, FR, Limoges/Bellegarde
4070, FR-N, Toulouse-Blagnac Airport, 43.629101, 1.36382, EU, FR, Toulouse/Blagnac
4071, FR-B, Pau Pyrénées Airport, 43.380001068115234, -0.41861099004745483, EU, FR, Pau/Pyrénées (Uzein)
4073, FR-N, Tarbes-Lourdes-Pyrénées Airport, 43.1786994934082, -0.006438999902456999, EU, FR, Tarbes/Lourdes/Pyrénées
4074, FR-T, 

In [3]:
from py2neo import authenticate, Graph, Relationship
graph = Graph("http://localhost:7474/db/data/")
from py2neo import Node, Relationship
graph.delete_all()

In [4]:
NO_OF_DAYS = 15
days = [ Node("Date", date = i) for i in range (1,NO_OF_DAYS+1)]

processed_days = []

for day in days:
    graph.create( day )
    
    for pd in processed_days:
        relationship = Relationship (pd, "Later", day)
        days_between = day['date'] - pd['date']
        relationship['date'] = days_between
        graph.create( relationship )
        
    processed_days.append(day)

In [5]:
%%time
regions, countries, continents, cities = {}, {}, {}, {}

tx = graph.begin()

current_id = 1000

for index,row in enumerate(reader[1:]):
    #if index % 2000 == 0:
       # print row[3], row[8]
        
    if row[7] == 'EU' and row[2] == 'large_airport' and row[11] == 'yes':
        print ( ", ".join ( row[x] for x in [0,9,3,4,5,7,8,10,2]) )
        region_name = row[9]
        region = regions.get(region_name, None)
        if not region:
            region = Node("Region", region = region_name)
            regions[region_name] = region
            tx.create(region)
        
        country_name = row[8]
        country = countries.get(country_name, None)
        if not country:
            country = Node("Country", country = country_name)
            countries[country_name] = country
            tx.create(country)
            
        continent_name = row[7]
        continent = continents.get(continent_name, None)
        if not continent:
            continent = Node("Continent", continent = continent_name)
            continents[continent_name] = continent
            tx.create(continent)
            
        city_name = row[10].split("/")[0]
        city = cities.get(city_name, None)
        if not city:
            city = Node("City", city = city_name)
            cities[city_name] = city
            tx.create(city)
        
        airport_name = row[3]
        airport = Node("Airport", airport=airport_name)
        
        #ADD PROPERTIES
        airport['id'] = current_id
        airport['continent'] = continent_name
        airport['country'] = country_name
        airport['region'] = region_name
        airport['city'] = city_name
        
        tx.create(airport)
        tx.create(Relationship(airport, "IN_REGION", region))
        tx.create(Relationship(airport, "IN_COUNTRY", country))
        tx.create(Relationship(airport, "IN_CONTINENT", continent))
        tx.create(Relationship(airport, "IN_CITY", city))
        
        airport_days = []
        
        for day in days:
            number = day['date']
            airport_day = Node("AirportDay", airport = airport_name, day = number)
            
            for day in days[number:]:
                gap = day['date'] - number
                if gap > 0:
                    raise ValueError("Going Wrong Way!")
                    
                next_market = Relationship (airport_day, "NEXT_MARKET", day )
                next_market['days'] = days_betwee
                tx.create( )
                
            tx.create(airport_day)
            airport_rel = Relationship( airport_day, "HAS_AIRPORT", airport)
            day_rel = Relationship( airport_day, "HAS_DAY", day)
            tx.create(airport_rel)
            tx.create(day_rel)
            
            airport_days.append(airport_day)
            
        from itertools import combinations
        
        # moving in time
        for early, late in combinations(airport_days,2):
            days_between =   late ['day'] - early['day']
            if days_between < 0:
                early, late = late, early
                days_between = -days_between
                
            next_market = Relationship(early, "NEXT_MARKET", late)
            next_market['days'] = days_between
            tx.create(next_market)
            
        tx.process()
        
        current_id += 10
        
tx.commit()

123, IS-2, Keflavik International Airport, 63.985000610352, -22.605600357056, EU, IS, Reykjavík, large_airport
4614, XK-01, Priština International Airport, 42.5728, 21.035801, EU, XK, Prishtina, large_airport
2155, BE-BRU, Brussels Airport, 50.901401519800004, 4.48443984985, EU, BE, Brussels, large_airport
2157, BE-WHT, Brussels South Charleroi Airport, 50.459201812699995, 4.45382022858, EU, BE, Brussels, large_airport
2162, BE-WLG, Liège Airport, 50.63740158081055, 5.443220138549805, EU, BE, Liège, large_airport
2209, DE-BR, Berlin-Schönefeld International Airport, 52.380001068115, 13.522500038147, EU, DE, Berlin, large_airport
2210, DE-SN, Dresden Airport, 51.1328010559082, 13.767200469970703, EU, DE, Dresden, large_airport
2212, DE-HE, Frankfurt am Main International Airport, 50.026402, 8.54313, EU, DE, Frankfurt-am-Main, large_airport
2213, DE-NW, Münster Osnabrück Airport, 52.134601593, 7.68483018875, EU, DE, Münster, large_airport
2214, DE-HH, Hamburg Airport, 53.630401611328, 9.

Exception KeyError: (u'http://localhost:7474/db/data/relationship/319697',) in <function remove at 0x00000000086E9828> ignored
Exception KeyError: (u'http://localhost:7474/db/data/relationship/319698',) in <function remove at 0x00000000086E9828> ignored
Exception KeyError: (u'http://localhost:7474/db/data/relationship/319699',) in <function remove at 0x00000000086E9828> ignored
Exception KeyError: (u'http://localhost:7474/db/data/relationship/319700',) in <function remove at 0x00000000086E9828> ignored
Exception KeyError: (u'http://localhost:7474/db/data/relationship/319701',) in <function remove at 0x00000000086E9828> ignored
Exception KeyError: (u'http://localhost:7474/db/data/relationship/319702',) in <function remove at 0x00000000086E9828> ignored
Exception KeyError: (u'http://localhost:7474/db/data/relationship/319703',) in <function remove at 0x00000000086E9828> ignored


4344, IT-42, Genoa Cristoforo Colombo Airport, 44.41329956049999, 8.83749961853, EU, IT, Genova, large_airport
4345, IT-25, Linate Airport, 45.445098877, 9.27674007416, EU, IT, Milan, large_airport
4350, IT-21, Cuneo International Airport, 44.547000885, 7.623219966890001, EU, IT, Cuneo, large_airport
4354, IT-45, Bologna Guglielmo Marconi Airport, 44.5354, 11.2887, EU, IT, Bologna, large_airport
4355, IT-34, Treviso-Sant'Angelo Airport, 45.648399353, 12.1943998337, EU, IT, Treviso, large_airport
4366, IT-34, Verona Villafranca Airport, 45.395699, 10.8885, EU, IT, Verona, large_airport
4368, IT-34, Venice Marco Polo Airport, 45.505299, 12.3519, EU, IT, Venice, large_airport
4370, IT-62, Ciampino–G. B. Pastine International Airport, 41.7994003296, 12.5949001312, EU, IT, Roma, large_airport
4372, IT-62, Leonardo da Vinci–Fiumicino Airport, 41.8045005798, 12.2508001328, EU, IT, Rome, large_airport
4378, IT-72, Naples International Airport, 40.8860015869, 14.290800094600002, EU, IT, Nápoli,

In [6]:
%%time
# cre
t = graph.run("MATCH (d:Date {date:1}) <- [:HAS_DAY] - (ad:AirportDay) RETURN ad")
t = list(t)

Wall time: 348 ms


In [8]:




for number in range(1, NO_OF_DAYS+1):
    
    tx = graph.begin()
    query = "MATCH (d:Date {{date:{date}}}) <- [:HAS_DAY] - (ad:AirportDay) RETURN ad".format(date=number)
    ad_on_given_day = [item['ad'] for item in graph.run(query)]
    
    combs = combinations(ad_on_given_day, 2)
    
    # create flight
    # cost
    # home airport
    
    for origin, destination in combs:
        import random
        cost = random.randint(20,500)
        o,d = origin,destination
        return_cost = max(cost + random.randint(-50,50), random.randint(20,100))
        flight_out = Relationship(o,"FLIGHT", d, cost=cost, market=o['airport'], day=o['day'], destination=d['airport'])
        flight_in = Relationship(d, "FLIGHT", o, cost=return_cost, market=d['airport'], day=d['day'], destination=o['airport'])
        tx.create(flight_out)
        tx.create(flight_in)
        tx.process()
        
    print "\n===============\n"
    
    

    tx.commit()


# 

In [9]:
query = """
MATCH (airport1: Airport {country : 'IE'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
RETURN ad1
"""
for it in graph.run(query):
    print it

(u'ad1': (a23f322:AirportDay {airport:"Shannon Airport",day:10}))
(u'ad1': (c4721ed:AirportDay {airport:"Shannon Airport",day:11}))
(u'ad1': (a404972:AirportDay {airport:"Shannon Airport",day:8}))
(u'ad1': (ba5766d:AirportDay {airport:"Shannon Airport",day:9}))
(u'ad1': (c13b585:AirportDay {airport:"Shannon Airport",day:14}))
(u'ad1': (e0dc98f:AirportDay {airport:"Shannon Airport",day:15}))
(u'ad1': (e60b54c:AirportDay {airport:"Shannon Airport",day:12}))
(u'ad1': (aa20506:AirportDay {airport:"Shannon Airport",day:13}))
(u'ad1': (efc82e6:AirportDay {airport:"Shannon Airport",day:2}))
(u'ad1': (d1b5b2a:AirportDay {airport:"Shannon Airport",day:3}))
(u'ad1': (a271104:AirportDay {airport:"Shannon Airport",day:1}))
(u'ad1': (a36160d:AirportDay {airport:"Shannon Airport",day:6}))
(u'ad1': (cb5c6ba:AirportDay {airport:"Shannon Airport",day:7}))
(u'ad1': (d0365cf:AirportDay {airport:"Shannon Airport",day:4}))
(u'ad1': (e3a4500:AirportDay {airport:"Shannon Airport",day:5}))
(u'ad1': (d26803f:A

In [10]:
%%time
query = """
MATCH (airport1: Airport {country : 'IE'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay),
(airport2: Airport {continent : 'EU'} ) <- [:HAS_AIRPORT] - (ad2:AirportDay) <- [f1:FLIGHT] - (ad1)
RETURN ad1, ad2
"""

t = list(graph.run(query))
print len(t)

6840
Wall time: 39.8 s


In [11]:
%%time
query = """
MATCH (airport1: Airport {country : 'IE'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay),
(ad1) - [:FLIGHT] - > (ad2:AirportDay) - [:HAS_AIRPORT] -> (airport2: Airport {continent : 'EU'} )
RETURN ad1, ad2 
"""

t = list(graph.run(query))
print len(t)

6840
Wall time: 28 s


In [12]:
%%time
query = """
MATCH (airport1: Airport {country : 'IE'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay)- 
[f:FLIGHT] - > (ad2:AirportDay) - [:HAS_AIRPORT] -> (airport2: Airport {continent : 'EU'} )
RETURN ad1, ad2 
"""

t = list(graph.run(query))
print len(t)

6840
Wall time: 22.1 s


In [13]:
%%time
query = """
MATCH (airport1: Airport {country : 'IE'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay),
(airport2: Airport {continent : 'EU'} ) <- [:HAS_AIRPORT] - (ad2:AirportDay) <- [f:FLIGHT] - (ad1)
RETURN f
"""

t = list(graph.run(query))
print len(t)

for i,a in enumerate(t):
    if i % 200 == 100:
        print a

6840
(u'f': (a23f322)-[:FLIGHT {cost:224,day:10,destination:"Adolfo Suárez Madrid–Barajas Airport",market:"Shannon Airport"}]->(fd907aa))
(u'f': (c4721ed)-[:FLIGHT {cost:26,day:11,destination:11,market:"Shannon Airport"}]->(f648794))
(u'f': (ba5766d)-[:FLIGHT {cost:303,day:9,destination:9,market:"Shannon Airport"}]->(f80796f))
(u'f': (c13b585)-[:FLIGHT {cost:377,day:14,destination:14,market:"Shannon Airport"}]->(e4278a2))
(u'f': (e0dc98f)-[:FLIGHT {cost:455,day:15,destination:15,market:"Shannon Airport"}]->(c9a8e51))
(u'f': (f298560)-[:FLIGHT {cost:200,day:13,destination:13,market:"Shannon Airport"}]->(c3ce0c2))
(u'f': (c1529a6)-[:FLIGHT {cost:194,day:2,destination:2,market:"Shannon Airport"}]->(ce3defa))
(u'f': (e49af67)-[:FLIGHT {cost:251,day:3,destination:3,market:"Shannon Airport"}]->(f21f956))
(u'f': (d61100a)-[:FLIGHT {cost:267,day:6,destination:6,market:"Shannon Airport"}]->(fe6fe24))
(u'f': (fe487d3)-[:FLIGHT {cost:446,day:7,destination:7,market:"Shannon Airport"}]->(bdad39d))


In [14]:
%%time
query = """
MATCH (airport1: Airport {country : 'IE'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
WITH ad1
MATCH (ad1) - [f:FLIGHT] - > (ad2:AirportDay)  - [:HAS_AIRPORT] -> (airport2: Airport {continent : 'EU'} )
RETURN f 
"""

t = list(graph.run(query))
print len(t)

6840
Wall time: 29.6 s


In [16]:
%%time
query = """
MATCH (airport1: Airport {country : 'IE'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
WITH ad1,airport1
MATCH (ad1) - [f1:FLIGHT] - > (ad2:AirportDay)  - [:HAS_AIRPORT] -> (airport2: Airport {continent : 'EU'} ),
(ad2) - [nm:NEXT_MARKET {days: 8}] - (ad3),
(ad3) - [f2:FLIGHT] -> (ad4) - [:HAS_AIRPORT] -> (airport1: Airport {country : 'IE'} )
RETURN f1,f2
"""

t = list(graph.run(query))
print len(t)


6384
Wall time: 2min 38s


In [1]:
import pandas as pd

query = """
MATCH (airport1: Airport {country : 'IE'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
WITH ad1,airport1
MATCH (ad1) - [f1:FLIGHT] - > (ad2:AirportDay)  - [:HAS_AIRPORT] -> (airport2: Airport {country : 'FR'} ),
(ad2) - [nm:NEXT_MARKET {days: 12}] - (ad3),
(ad3) - [f2:FLIGHT] -> (ad4) - [:HAS_AIRPORT] -> (airport1: Airport {country : 'IE'} )
RETURN f1,f2
"""
results = list(graph.run(query))
df = pd.DataFrame(results)
df.head()

NameError: name 'graph' is not defined

In [4]:
test =['5','7+','9-','3 7']

def date_to_list(date):
    try:
        d = int(date)
        return [d, None]
    except:
        pass
    if '-' in date: return [date.split('-')[0], '<=']
    if '+' in date: return [date.split('+')[0], '>=']
    
    return date.split()
    
for t in test:
    print t, date_to_list(t)

5 [5, None]
7+ ['7', '>=']
9- ['9', '<=']
3 7 ['3', '7']
